# Competition-Colonization Tradeoffs

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import ipywidgets as widgets
%matplotlib widget

## What is it and how do we implement it?
The basic idea here is that some species may be less fit in terms of reproduction than their competitors but better at migration. We can represent this in our increasingly complex model by adding a sort of 'fitness' when we chose which individual migrates.

Before we were assuming a probability of selecting species 1 for migration equal to the frequency of species 1 in the entire population. Now we have to consider the migration fitness ratio as well ($k_1^m / k_2^m$)
$$
P_m(S=1) = \frac{(k_1^m / k_2^m)f_1^M}{(k_1^m / k_2^m)f_1^M + f_2^M}
$$

This is exactly the same function that we used for the probability of reproduction but with frequencies for the entire metacommunity and fitnesses for migration rather than reproduction.

In [8]:
def moran_competition_colonization(n_indiv, n_years, init_f1, avg_fit_ratio1, avg_fit_ratio2, freq_dep, fit_ratio_m, dispersal):
    """ Implements the Moran model including constant selection and simulates it in time. 
        
        Parameters
        ----------
        n_indiv: int
            number of individuals in the community
        n_years: int
            number of potential 'turnovers' of the community
        init_f1: float
            initial frequency of species 1
        avg_fit_ratio1: float
            fitness ratio of species 1 and 2 when both are in equal frequency for patch 1
        avg_fit_ratio2: float
            fitness ratio of species 1 and 2 when both are in equal frequency for patch 2
        freq_dep: float
            Strength of frequency dependence for selection
        fit_ratio_m: float
            Dispersal ability ratio between species 1 and species 2
        dispersal: float
            probability that a new birth is drawn from metacommunity
            
        Return
        ------
        moran: np.array (n_years, 2)
            contains the species frequencies for each year of the simulation """

    # For this we're going to track only species 1 and instead of the columns representing
    # different species they will be the two different patches
    m_patches = 2
    moran = np.zeros((n_years, m_patches))
    moran[0] = np.array(init_f1)

    # Now our community vector is instead a metacommunity matrix
    meta = np.random.choice([0, 1], size=(m_patches, n_indiv), replace=True, p=moran[0])

    # we're going to stick the fit ratios into a list so we can access them by index
    avg_fit_ratio = [avg_fit_ratio1, avg_fit_ratio2]

    # now we can do the loop as in vellend. I'm going to write it a bit differently
    # but the idea is basically the same. (yes its slower)
    for year in range(1, n_years):

        # we now need to check each patch as well as each year
        for patch in range(m_patches):

            # for each year in each patch we do replacement for every individual
            for indiv in range(n_indiv):

                # select patch where death occurs
                death_patch = np.random.choice(range(m_patches))

                # now every time we replace an individual we need to check whether or not
                # dispersal occurs. Draw from entire metacommunity.
                # Now we also consider a sort of fitness for migration
                if np.random.uniform() < dispersal:
                    meta_f1 = np.sum(meta == 0) / (n_indiv * m_patches)
                    pr1 = fit_ratio_m * meta_f1 / (fit_ratio_m * meta_f1 + (1 - meta_f1))
                # or draw from local community using our frequency dependent selection
                else:
                    f1 = np.sum(meta[death_patch] == 0) / n_indiv
                    fit_ratio = avg_fit_ratio[death_patch] * np.exp(freq_dep*(f1 - 0.5))
                    pr1 = fit_ratio * f1 / (fit_ratio * f1 + (1 - f1))

                # replace one individual with another
                death = np.random.randint(n_indiv)
                birth = np.random.choice([0, 1], p=[pr1, 1 - pr1])
                meta[death_patch, death] = birth

            # when we're done looping over all of the individuals we can update
            # frequencies for the year (the timescale we care about)
            f1 = np.sum(meta[patch] == 0) / n_indiv
            moran[year, patch] = f1

    return moran

In [9]:
def draw_simulation(iterations=10, individuals=100, years=50, initial_freq=0.5, avg_fit_ratio1=1.0, avg_fit_ratio2=1.0, freq_dep=0.0, fit_ratio_m=1.0, dispersal=0):
    # the plot bit, this just makes a blank figure
    fig, ax = plt.subplots(ncols=3, figsize=(10,4), gridspec_kw = {'width_ratios':[2, 3, 1]})
    # plot the frequency dependence as in Ch. 6
    # frequency range and neutral line as above
    f1 = np.arange(0, 1, 0.01)
    ax[0].axhline(1, linestyle='--', color='grey', alpha=0.4)
    # dependence curve
    ax[0].plot(f1, avg_fit_ratio1 * np.exp(freq_dep*(f1 - 0.5)))
    ax[0].plot(f1, avg_fit_ratio2 * np.exp(freq_dep*(f1 - 0.5)))
    ax[0].set_xlabel('Frequency of species 1')
    ax[0].set_ylabel('Fitness ratio 1:2')

    # trajectory labels
    ax[1].set_xlabel('Years')
    ax[1].set_ylabel('Species 1 frequency')
    ax[1].set_ylim([0, 1.01])
    # distribution labels and bins
    hist_bins = np.arange(0, 1.1, 0.1)
    ax[2].set_xlabel('Count')
    ax[2].set_xlim([0, iterations])
    ax[2].set_ylim([0, 1.01])

    # we're going to plot a distribution too need an array for it
    dist1 = np.zeros(iterations)
    dist2 = np.zeros(iterations)


    # we run the simulation and draw a trajectory
    for i in tqdm(range(iterations)):
        simulation = moran_competition_colonization(individuals, years, initial_freq, avg_fit_ratio1, avg_fit_ratio2, freq_dep, fit_ratio_m, dispersal)
        ax[1].plot(range(years), simulation[:, 0], c='C0')
        ax[1].plot(range(years), simulation[:, 1], c='C1')

        # add final freq to our dist array
        dist1[i] = simulation[-1, 0]
        dist2[i] = simulation[-1, 1]

    # plot the distribution too
    ax[2].hist(dist1, bins=hist_bins, orientation='horizontal', alpha=0.5, color='C0')
    ax[2].hist(dist2, bins=hist_bins, orientation='horizontal', alpha=0.5, color='C1')
    ax[2].axhline(np.mean(dist1), linestyle='--', c='C0')
    ax[2].axhline(np.mean(dist2), linestyle='--', c='C1')



    plt.tight_layout()    
    plt.show()

# set up the interface
widgets.interact_manual(draw_simulation, iterations=(1, 16, 5), individuals=(10, 500, 10), years=(5, 200, 5),
                  initial_freq=(0.0, 1.0, 0.01), avg_fit_ratio1=(0.67, 1.5, 0.01), avg_fit_ratio2=(0.67, 1.5, 0.01),
                   freq_dep=(-1, 1, 0.01), fit_ratio_m=(0.67, 1.5, 0.01), dispersal=(0, 1, 0.01))

interactive(children=(IntSlider(value=10, description=&#39;iterations&#39;, max=16, min=1, step=5), IntSlider(value=10…

&lt;function __main__.draw_simulation(iterations=10, individuals=100, years=50, initial_freq=0.5, avg_fit_ratio1=1.0, avg_fit_ratio2=1.0, freq_dep=0.0, fit_ratio_m=1.0, dispersal=0)&gt;